# Импорты

In [ ]:
import pandas as pd
import numpy as np
from dataprep.eda import create_report
# from google.colab import files
import os
# files.upload()

# Загрузка файлов

In [ ]:
def loading(path: str, name: str) -> pd.DataFrame:
    """
    this function loading crypto index and keep only closed values
    args:
        - path: path to file
        - name: name of index
    return:
        - pandas DataFrame with columns Date and index
    """

    columns_to_change = ['Close', 'Volume', 'Market Cap']
    renaming_dict = {
        'Date': 'date', 'Close': name,
        'Volume': name + '_vol', 'Market Cap': name + '_cap',
        'Vol Chg': name + '_val_chg' , 'MCap Chg': name + '_cap_chg'
    }

    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

    for column in columns_to_change:
        df[column] = df[column].str.replace('$', '', regex=False)
        df[column] = df[column].str.replace(',', '', regex=False)
        df[column] = df[column].astype('double')

    df = df.rename(columns=renaming_dict)
    df = df[renaming_dict.values()]
    for col in df.columns[1:]:
        try:
            df[col] = df[col].str.replace(',', '', regex=False)
            df[col] = df[col].astype('double')
        except AttributeError:
            continue

    return df

In [ ]:
directory = 'input'
crypto_data = {};
crypto_df = pd.DataFrame()
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        name = filename[:-4]
        df = loading(f, name)
        if crypto_df.empty:
            crypto_df = df
        else:
            crypto_df = crypto_df.merge(df, on='date', how='outer')
crypto_df = crypto_df.sort_values('date').reset_index(drop=True)
crypto_df = crypto_df.set_index('date')

# Dataprep

In [ ]:
report = create_report(crypto_df)
report.save('crypto_report.html')


Report has been saved to bitcoin_report.html!


#Pandas profiling

In [10]:
# !pip3 install  pandas-profiling --upgrade Pillow

In [ ]:
from ydata_profiling import ProfileReport
report = ProfileReport(crypto_df)
report.to_file('profile_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Sweetviz

In [9]:
# !pip3 install sweetviz

In [ ]:
import sweetviz as sv
report_sv = sv.analyze(crypto_df)
report_sv.show_html("report_sweetviz.html")

                                             |          | [  0%]   00:00 -> (? left)

Report report_sweetviz.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
